In [1]:
import pandas as pd
import numpy as np
import re
import math
import seaborn as sns
from matplotlib.pyplot import figure

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import mglearn
from IPython.display import display
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error

Read excel

In [2]:
M=pd.read_excel("/Users/tingyinglu/Desktop/research/rawdata.xlsx",sheet_name='Sheet1', engine='openpyxl', header=0).dropna(how='all')

In [3]:
M

,#,gene,Ct1,Ct2,Ct3,Mean,Treshold,date,group,person,experiment#,HK(ACTB) date,HK(ACTB) Ct,dCt (ACTB)
0,73R,YWHAZ,29.680000,29.530000,29.920000,29.71,1.629,`010522,(G03) 6w_XX_BSO-T,YT,6.0,`092321,24.84,4.87
1,74L,YWHAZ,31.050000,30.600000,31.030000,30.89,1.629,`010522,(G03) 6w_XX_BSO-T,YT,6.0,`092321,24.08,6.81
2,104L,YWHAZ,30.350000,30.020000,29.890000,30.09,1.629,`010522,(G03) 6w_XX_BSO-T,YT,6.0,`072921,25.41,4.67
3,104R,YWHAZ,30.450000,29.440000,30.060000,29.98,1.629,`010522,(G03) 6w_XX_BSO-T,YT,6.0,`072921,25.55,4.43
4,92L,YWHAZ,NaN,28.450000,28.490000,28.47,1.629,`010522,(G04) 6w_XX_BSO+T,YT,6.0,`080521,23.81,4.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442,140L,CXCL1,27.941544,28.064907,27.860519,27.96,NaN,`031022,(G05) 6w_XX_BSO+2T,NaN,8.0,`030122,20.42,7.54
1443,140L,IFNG,31.800512,29.822184,31.309832,30.98,NaN,`031022,(G05) 6w_XX_BSO+2T,NaN,8.0,`030122,20.42,10.56
1444,140R,TGFB1,30.000000,NaN,NaN,30.00,NaN,`031022,(G05) 6w_XX_BSO+2T,NaN,8.0,`030122,24.03,5.97
1445,140R,CXCL1,32.490242,34.826889,34.531269,33.95,NaN,`031022,(G05) 6w_XX_BSO+2T,NaN,8.0,`030122,24.03,9.92


See how many gene we have.

In [5]:
M['gene'].unique()

array(['YWHAZ', 'HPRT', 'RPL13A', 'EEF1A1', 'ACTB', 'COL-1', 'TNFA',
       'F480', 'EEF2', 'NOS2', 'ARG1', 'CD3', 'ASMA', 'PDGFA', 'IL6',
       'VEGFA', 'CSF1', 'IGF1', 'FGF1', 'PDGFa', 'COL-3A1', 'EGF', 'KI67',
       'EMR1', 'AR', 'FN1', 'FOXP3', 'CD4', 'TGFB1', 'MIF', 'CXCL1',
       'IFNG', 'MMP13'], dtype=object)

change raw data format

In [7]:
M1=pd.DataFrame(M.groupby(['#','gene'])['dCt (ACTB)'].mean().unstack().copy())

In [8]:
M1['mouse']=M1.index.copy()

M1['group']=np.nan
M1


gene,ACTB,AR,ARG1,ASMA,CD3,CD4,COL-1,COL-3A1,CSF1,CXCL1,...,NOS2,PDGFA,PDGFa,RPL13A,TGFB1,TNFA,VEGFA,YWHAZ,mouse,group
#,,,,,,,,,,,,,,,,,,,,,
100L,0.0,NaN,NaN,2.22,NaN,NaN,-1.47,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.24,NaN,NaN,100L,NaN
100R,0.0,NaN,NaN,4.75,NaN,NaN,5.12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.81,NaN,NaN,100R,NaN
101L,0.0,NaN,NaN,2.36,NaN,NaN,-1.03,NaN,0.13,NaN,...,NaN,0.12,NaN,NaN,NaN,8.10,5.89,NaN,101L,NaN
101R,0.0,NaN,NaN,1.36,NaN,NaN,-0.69,NaN,4.12,NaN,...,NaN,4.37,NaN,NaN,NaN,9.80,6.20,NaN,101R,NaN
102L,0.0,NaN,NaN,2.74,NaN,NaN,0.44,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,15.89,NaN,NaN,102L,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97R,0.0,NaN,NaN,-0.83,NaN,NaN,-2.16,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.00,NaN,NaN,97R,NaN
98L,0.0,NaN,NaN,0.70,NaN,NaN,-2.06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.08,NaN,NaN,98L,NaN
98R,0.0,NaN,NaN,6.12,NaN,NaN,7.15,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.22,NaN,NaN,98R,NaN


In [9]:
M1.columns

Index(['ACTB', 'AR', 'ARG1', 'ASMA', 'CD3', 'CD4', 'COL-1', 'COL-3A1', 'CSF1',
       'CXCL1', 'EEF1A1', 'EEF2', 'EGF', 'EMR1', 'F480', 'FGF1', 'FN1',
       'FOXP3', 'HPRT', 'IFNG', 'IGF1', 'IL6', 'KI67', 'MIF', 'MMP13', 'NOS2',
       'PDGFA', 'PDGFa', 'RPL13A', 'TGFB1', 'TNFA', 'VEGFA', 'YWHAZ', 'mouse',
       'group'],
      dtype='object', name='gene')

In [10]:
for i in M1['mouse']:
    print(i)
    print(M['group'].loc[M['#']==i].unique())
    M1['group'].loc[M1['mouse']==i]=M['group'].loc[M['#']==i].unique().copy()

100L
['(G10) 6w_XY_GO+2T']
100R
['(G10) 6w_XY_GO+2T']
101L
['(G01) 6w_XX_SHAM-T']
101R
['(G01) 6w_XX_SHAM-T']
102L
['(G01) 6w_XX_SHAM-T']
102R
['(G01) 6w_XX_SHAM-T']
103L
['(G03) 6w_XX_BSO-T']
103R
['(G03) 6w_XX_BSO-T']
104L
['(G03) 6w_XX_BSO-T']
104R
['(G03) 6w_XX_BSO-T']
105L
['(G04) 6w_XX_BSO+T']
105R
['(G04) 6w_XX_BSO+T']
106L
['(G04) 6w_XX_BSO+T']
106R
['(G04) 6w_XX_BSO+T']
107L
['(G05) 6w_XX_BSO+2T']
107R
['(G05) 6w_XX_BSO+2T']
108L
['(G05) 6w_XX_BSO+2T']
108R
['(G05) 6w_XX_BSO+2T']
109L
['(G08) 6w_XY_GO-T']
109R
['(G08) 6w_XY_GO-T']
110L
['(G08) 6w_XY_GO-T']
110R
['(G08) 6w_XY_GO-T']
111L
['(G10) 6w_XY_GO+2T']
111R
['(G10) 6w_XY_GO+2T']
112L
['(G10) 6w_XY_GO+2T']
112R
['(G10) 6w_XY_GO+2T']
113L
['(G01) 6w_XX_SHAM-T']
113R
['(G01) 6w_XX_SHAM-T']
114L
['(G01) 6w_XX_SHAM-T']
114R
['(G01) 6w_XX_SHAM-T']
115L
['(G03) 6w_XX_BSO-T']
115R
['(G03) 6w_XX_BSO-T']
116L
['(G03) 6w_XX_BSO-T']
116R
['(G03) 6w_XX_BSO-T']
117L
['(G04) 6w_XX_BSO+T']
117R
['(G04) 6w_XX_BSO+T']
118L
['(G04) 6w_XX_B

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)



['(G05) 6w_XX_BSO+2T']
145L
['(G01) 6w_XX_SHAM-T']
145R
['(G01) 6w_XX_SHAM-T']
147L
['(G03) 6w_XX_BSO-T']
147R
['(G03) 6w_XX_BSO-T']
152L
['(G05) 6w_XX_BSO+2T']
152R
['(G05) 6w_XX_BSO+2T']
153R
['(G05) 6w_XX_BSO+2T']
154L
['(G05) 6w_XX_BSO+2T']
158L
['(G03) 6w_XX_BSO-T']
158R
['(G03) 6w_XX_BSO-T']
159L
['(G03) 6w_XX_BSO-T']
159R
['(G03) 6w_XX_BSO-T']
160L
['(G04) 6w_XX_BSO+T']
160R
['(G04) 6w_XX_BSO+T']
161L
['(G05) 6w_XX_BSO+2T']
161R
['(G05) 6w_XX_BSO+2T']
170L
['(G03) 6w_XX_BSO-T']
170R
['(G03) 6w_XX_BSO-T']
171L
['(G03) 6w_XX_BSO-T']
171R
['(G03) 6w_XX_BSO-T']
173L
['(G05) 6w_XX_BSO+2T']
173R
['(G05) 6w_XX_BSO+2T']
180L
['(G10) 6w_XY_GO+2T']
180R
['(G10) 6w_XY_GO+2T']
73L
['(G03) 6w_XX_BSO-T']
73R
['(G03) 6w_XX_BSO-T']
74L
['(G03) 6w_XX_BSO-T']
74R
['(G03) 6w_XX_BSO-T']
75L
['(G03) 6w_XX_BSO-T']
75R
['(G03) 6w_XX_BSO-T']
79L
['(G05) 6w_XX_BSO+2T']
79R
['(G05) 6w_XX_BSO+2T']
80L
['(G05) 6w_XX_BSO+2T']
80R
['(G05) 6w_XX_BSO+2T']
81L
['(G05) 6w_XX_BSO+2T']
81R
['(G05) 6w_XX_BSO+2T']


In [11]:
M1

gene,ACTB,AR,ARG1,ASMA,CD3,CD4,COL-1,COL-3A1,CSF1,CXCL1,...,NOS2,PDGFA,PDGFa,RPL13A,TGFB1,TNFA,VEGFA,YWHAZ,mouse,group
#,,,,,,,,,,,,,,,,,,,,,
100L,0.0,NaN,NaN,2.22,NaN,NaN,-1.47,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.24,NaN,NaN,100L,(G10) 6w_XY_GO+2T
100R,0.0,NaN,NaN,4.75,NaN,NaN,5.12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.81,NaN,NaN,100R,(G10) 6w_XY_GO+2T
101L,0.0,NaN,NaN,2.36,NaN,NaN,-1.03,NaN,0.13,NaN,...,NaN,0.12,NaN,NaN,NaN,8.10,5.89,NaN,101L,(G01) 6w_XX_SHAM-T
101R,0.0,NaN,NaN,1.36,NaN,NaN,-0.69,NaN,4.12,NaN,...,NaN,4.37,NaN,NaN,NaN,9.80,6.20,NaN,101R,(G01) 6w_XX_SHAM-T
102L,0.0,NaN,NaN,2.74,NaN,NaN,0.44,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,15.89,NaN,NaN,102L,(G01) 6w_XX_SHAM-T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97R,0.0,NaN,NaN,-0.83,NaN,NaN,-2.16,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.00,NaN,NaN,97R,(G08) 6w_XY_GO-T
98L,0.0,NaN,NaN,0.70,NaN,NaN,-2.06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.08,NaN,NaN,98L,(G08) 6w_XY_GO-T
98R,0.0,NaN,NaN,6.12,NaN,NaN,7.15,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.22,NaN,NaN,98R,(G08) 6w_XY_GO-T


See all mouse

In [14]:
print(M1['mouse'].values)

['100L' '100R' '101L' '101R' '102L' '102R' '103L' '103R' '104L' '104R'
 '105L' '105R' '106L' '106R' '107L' '107R' '108L' '108R' '109L' '109R'
 '110L' '110R' '111L' '111R' '112L' '112R' '113L' '113R' '114L' '114R'
 '115L' '115R' '116L' '116R' '117L' '117R' '118L' '118R' '119L' '119R'
 '120L' '120R' '121L' '121R' '123L' '123R' '124L' '124R' '127L' '127R'
 '128L' '128R' '133L' '133R' '134L' '134R' '135L' '135R' '136L' '136R'
 '139L' '139R' '140L' '140R' '145L' '145R' '147L' '147R' '152L' '152R'
 '153R' '154L' '158L' '158R' '159L' '159R' '160L' '160R' '161L' '161R'
 '170L' '170R' '171L' '171R' '173L' '173R' '180L' '180R' '73L' '73R' '74L'
 '74R' '75L' '75R' '79L' '79R' '80L' '80R' '81L' '81R' '85L' '85R' '86L'
 '86R' '87L' '87R' '88L' '88R' '91L' '91R' '92L' '92R' '93L' '93R' '94L'
 '94R' '95L' '95R' '97L' '97R' '98L' '98R' '99L' '99R']


Choose a mouse to do normalization. Here we choose 135L

In [15]:
x='135L'
M1.loc[M1['mouse']==x]

gene,ACTB,AR,ARG1,ASMA,CD3,CD4,COL-1,COL-3A1,CSF1,CXCL1,...,NOS2,PDGFA,PDGFa,RPL13A,TGFB1,TNFA,VEGFA,YWHAZ,mouse,group
#,,,,,,,,,,,,,,,,,,,,,
135L,0.0,6.93,3.84,1.29,14.01,13.35,-0.82,-1.31,NaN,12.53,...,8.36,NaN,NaN,NaN,11.43,10.74,7.45,7.4,135L,(G03) 6w_XX_BSO-T


In [17]:
#choose control group. for example: 135L
col=['ACTB', 'AR', 'ARG1', 'ASMA', 'CD3', 'CD4', 'COL-1', 'COL-3A1', 'CSF1',
       'CXCL1', 'EEF1A1', 'EEF2', 'EGF', 'EMR1', 'F480', 'FGF1', 'FN1',
       'FOXP3', 'HPRT', 'IFNG',  'IGF1', 'IL6', 'KI67', 'MIF', 'MMP13',
       'NOS2', 'PDGFA', 'PDGFa', 'RPL13A', 'TGFB1',  'TNFA', 'VEGFA',
        'YWHAZ'  ]
M2=M1[col].copy()
M3=M1[col].loc[M1['mouse']==x].copy()
print(M3)


gene  ACTB    AR  ARG1  ASMA    CD3    CD4  COL-1  COL-3A1  CSF1  CXCL1  ...  \
#                                                                        ...   
135L   0.0  6.93  3.84  1.29  14.01  13.35  -0.82    -1.31   NaN  12.53  ...   

gene  MIF  MMP13  NOS2  PDGFA  PDGFa  RPL13A  TGFB1   TNFA  VEGFA  YWHAZ  
#                                                                         
135L  8.7  10.73  8.36    NaN    NaN     NaN  11.43  10.74   7.45    7.4  

[1 rows x 33 columns]


normalization part

In [18]:


M2=M2.sub(M3.values, axis='columns')

In [19]:
M2[['mouse', 'group']]=M1[['mouse', 'group']].copy()

In [20]:
M2

gene,ACTB,AR,ARG1,ASMA,CD3,CD4,COL-1,COL-3A1,CSF1,CXCL1,...,NOS2,PDGFA,PDGFa,RPL13A,TGFB1,TNFA,VEGFA,YWHAZ,mouse,group
#,,,,,,,,,,,,,,,,,,,,,
100L,0.0,NaN,NaN,0.93,NaN,NaN,-0.65,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.50,NaN,NaN,100L,(G10) 6w_XY_GO+2T
100R,0.0,NaN,NaN,3.46,NaN,NaN,5.94,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.07,NaN,NaN,100R,(G10) 6w_XY_GO+2T
101L,0.0,NaN,NaN,1.07,NaN,NaN,-0.21,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-2.64,-1.56,NaN,101L,(G01) 6w_XX_SHAM-T
101R,0.0,NaN,NaN,0.07,NaN,NaN,0.13,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.94,-1.25,NaN,101R,(G01) 6w_XX_SHAM-T
102L,0.0,NaN,NaN,1.45,NaN,NaN,1.26,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.15,NaN,NaN,102L,(G01) 6w_XX_SHAM-T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97R,0.0,NaN,NaN,-2.12,NaN,NaN,-1.34,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-8.74,NaN,NaN,97R,(G08) 6w_XY_GO-T
98L,0.0,NaN,NaN,-0.59,NaN,NaN,-1.24,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-4.66,NaN,NaN,98L,(G08) 6w_XY_GO-T
98R,0.0,NaN,NaN,4.83,NaN,NaN,7.97,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.52,NaN,NaN,98R,(G08) 6w_XY_GO-T


fold change

In [22]:
M2[col]=M2[col].transform(lambda x: 2**(-x))

In [24]:
M2

gene,ACTB,AR,ARG1,ASMA,CD3,CD4,COL-1,COL-3A1,CSF1,CXCL1,...,NOS2,PDGFA,PDGFa,RPL13A,TGFB1,TNFA,VEGFA,YWHAZ,mouse,group
#,,,,,,,,,,,,,,,,,,,,,
100L,1.0,NaN,NaN,0.524858,NaN,NaN,1.569168,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.414214,NaN,NaN,100L,(G10) 6w_XY_GO+2T
100R,1.0,NaN,NaN,0.090873,NaN,NaN,0.016289,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.952638,NaN,NaN,100R,(G10) 6w_XY_GO+2T
101L,1.0,NaN,NaN,0.476319,NaN,NaN,1.156688,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.233317,2.948538,NaN,101L,(G01) 6w_XX_SHAM-T
101R,1.0,NaN,NaN,0.952638,NaN,NaN,0.913831,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.918528,2.378414,NaN,101R,(G01) 6w_XX_SHAM-T
102L,1.0,NaN,NaN,0.366021,NaN,NaN,0.417544,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.028164,NaN,NaN,102L,(G01) 6w_XX_SHAM-T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97R,1.0,NaN,NaN,4.346939,NaN,NaN,2.531513,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,427.565015,NaN,NaN,97R,(G08) 6w_XY_GO-T
98L,1.0,NaN,NaN,1.505247,NaN,NaN,2.361985,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,25.281322,NaN,NaN,98L,(G08) 6w_XY_GO-T
98R,1.0,NaN,NaN,0.035158,NaN,NaN,0.003988,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.433955,NaN,NaN,98R,(G08) 6w_XY_GO-T


visualization part
Here we choose ('TNFA','VEGFA','TGFB1','MIF','CXCL1','IFNG','IL6','COL-1','COL-3A1','ASMA','FN1','F480','ARG1','NOS2','FOXP3','CD4','AR' ) those genes to do visualization

In [30]:

col1=['mouse','group','TNFA','VEGFA','TGFB1','MIF','CXCL1','IFNG','IL6','COL-1','COL-3A1','ASMA','FN1','F480','ARG1','NOS2','FOXP3','CD4','AR']
final_M2=M2[col1].dropna().copy()

In [26]:
final_M2

gene,mouse,group,TNFA,VEGFA,TGFB1,MIF,CXCL1,IFNG,IL6,COL-1,COL-3A1,ASMA,FN1,F480,ARG1,NOS2,FOXP3,CD4,AR
#,,,,,,,,,,,,,,,,,,,
115L,115L,(G03) 6w_XX_BSO-T,0.389582,2.428390,59.301636,6.105037,380.038034,1734.134400,101.125288,0.274523,0.198884,0.168404,3.810552,2.173470,0.075625,1.531558,13.928809,398.932265,87.426576
115R,115R,(G03) 6w_XX_BSO-T,0.632878,0.901250,44.017338,11.631780,165.421162,2194.992051,151.167061,0.028360,0.010972,0.068631,3.271608,2.445281,0.013230,4.407620,8.224911,975.501310,157.586485
116L,116L,(G03) 6w_XX_BSO-T,0.178006,4.228072,48.502930,3.810552,164.278515,2740.075632,101.828670,0.861546,0.158220,0.500000,3.204280,0.668964,0.051832,2.907945,20.534814,138.141214,49.866533
116R,116R,(G03) 6w_XX_BSO-T,0.063813,0.768438,19.698311,3.482202,49.180006,910.174906,131.598570,0.362236,0.071298,0.165511,1.558329,0.102949,0.010380,2.828427,6.821079,119.428223,59.714111
121L,121L,(G03) 6w_XX_BSO-T,0.007546,9.646463,0.594604,1.827663,11.631780,2.584706,10.556063,1.979313,1.164734,0.395021,0.795536,1.404445,1.469169,5.426417,0.006172,2.329467,1.109569
121R,121R,(G03) 6w_XX_BSO-T,1.351911,2.789487,0.615572,0.840896,2.128740,3.655326,10.196485,1.132884,0.835088,0.598739,0.582367,0.858565,0.091505,0.637280,0.019915,10.777869,2.158456
135L,135L,(G03) 6w_XX_BSO-T,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
135R,135R,(G03) 6w_XX_BSO-T,0.763130,2.203810,0.327598,0.371131,9.063071,8.938297,4.958831,0.266093,1.624505,0.171943,0.726986,0.664343,0.444421,3.972370,0.112656,0.216134,0.220676
136L,136L,(G03) 6w_XX_BSO-T,0.123279,8.168097,33.824577,20.252106,139.102062,3125.778860,79.893155,0.134904,0.176777,0.069348,0.314253,1.222640,0.395021,3.863745,17.148375,149.085899,38.854236


change data format for visualization.

In [27]:
g1=final_M2.melt(id_vars=['mouse','group'],value_vars=['TNFA','VEGFA','TGFB1','MIF','CXCL1','IFNG','IL6','COL-1','COL-3A1','ASMA','FN1','F480','ARG1','NOS2','FOXP3','CD4','AR'],var_name='gene',value_name='value')

In [28]:
g1

,mouse,group,gene,value
0,115L,(G03) 6w_XX_BSO-T,TNFA,0.389582
1,115R,(G03) 6w_XX_BSO-T,TNFA,0.632878
2,116L,(G03) 6w_XX_BSO-T,TNFA,0.178006
3,116R,(G03) 6w_XX_BSO-T,TNFA,0.063813
4,121L,(G03) 6w_XX_BSO-T,TNFA,0.007546
...,...,...,...,...
403,154L,(G05) 6w_XX_BSO+2T,AR,5.314743
404,79L,(G05) 6w_XX_BSO+2T,AR,128.444384
405,79R,(G05) 6w_XX_BSO+2T,AR,110.277908
406,80R,(G05) 6w_XX_BSO+2T,AR,120.676421


In [29]:
sns.set(font_scale = 10)

sns.set_style("ticks")



sns.catplot(x="gene", y="value",hue='group', data=g1, kind="box", 
            height=65, aspect=2.5,showfliers = False)

sns.stripplot(x="gene", y="value",hue='group', data=g1,jitter=0.3, dodge=True, marker='o', palette="Set2", alpha=1,s = 40)
plt.legend([],[], frameon=False)
plt.yscale('log')